<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/Zako_AI_Lessons/blob/main/DataAnalysis_W1_6_Pandas(DataStructures_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zako-ai

# Pandas Ma'lumot Strukturalari Umumiy Ko'rinishi  
Ushbu bo'limda biz `Series`, `Index` va `DataFrame` klasslari haqida gaplashamiz. Buni amalga oshirish uchun keyinchalik ishlatadigan CSV faylidan bir bo'lagini o'qib olamiz.
## Ma'lumotlar haqida  
Ushbu daftarda biz 2018-yil 18-sentabrdan 13-oktabrgacha bo'lgan zilzilalar ma'lumotlaridan 5 ta qatordan foydalanamiz (AQSh Geologiya Xizmatining [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/) orqali olingan).

## NumPy Arrays bilan Ishlash  
Keling, ba'zi namunaviy ma'lumotlar uchun qisqa CSV faylini o'qiymiz (buning uchun `numpy`ni ishlatamiz).  


In [ ]:
import numpy as np

data = np.genfromtxt(
    '../data/example_data.csv', delimiter=';',
    names=True, dtype=None, encoding='UTF'
)
data

array([('2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia', 'mww', 6.7, 'green', 1),
       ('2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww', 5.2, 'green', 0),
       ('2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww', 5.7, 'green', 0),
       ('2018-10-12 21:09:49.240', '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0),
       ('2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea', 'mww', 5.6, 'green', 1)],
      dtype=[('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

ma'lumotlar shaklini `shape` orqali ifodalash

In [ ]:
data.shape

(5,)

ma'lumot turini `dtype` orqali ifodalash

In [ ]:
data.dtype

dtype([('time', '<U23'), ('place', '<U37'), ('magType', '<U3'), ('mag', '<f8'), ('alert', '<U5'), ('tsunami', '<i4')])

Har bir kirish massivdagi qator CSV faylidan olingan. NumPy massivlari bitta ma'lumot turini o'z ichiga oladi (ro'yxatlar esa aralash turlarni qo'llab-quvvatlaydi); bu tezkor, vektorlashtirilgan operatsiyalarni bajarishga imkon beradi. Ma'lumotlarni o'qiganimizda, biz `numpy.void` obyektlaridan tashkil topgan massivni oldik, bu obyektlar moslashuvchan turlarni saqlash uchun yaratilgan. Buning sababi shundaki, NumPy har bir qator uchun bir nechta turli ma'lumot turlarini saqlashi kerak: to'rtta satr, biron-bir suzuvchi nuqta va bir butun son. Bu esa bitta ma'lumot turidagi obyektlar uchun NumPy taqdim etadigan ishlash samaradorligini foydalana olishimizga to'sqinlik qiladi.

Agar biz maksimal magnitudani topmoqchi bo'lsak, har bir qatorning uchinchi indeksini tanlash uchun **[list comprehension](https://www.python.org/dev/peps/pep-0202/)** dan foydalanishimiz mumkin, bu esa `numpy.void` obyektini bildiradi. Bu ro'yxatni yaratadi, demak biz `max()` funksiyasini ishlatib maksimal qiymatni topishimiz mumkin:


In [ ]:
%%timeit
max([row[3] for row in data])

2.77 µs ± 145 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Agar biz, o'rniga, har bir ustun uchun NumPy massivini yaratadigan bo'lsak, bu operatsiyani bajarish ancha oson (va samaraliroq) bo'ladi. Biz **[dictionary comprehension](https://www.python.org/dev/peps/pep-0274/)** yordamida ustun nomlari kalit bo'lib, ma'lumotlarning NumPy massivlari qiymat bo'lgan lug'at yaratishimiz mumkin:


In [ ]:
array_dict = {
    col: np.array([row[i] for row in data])
    for i, col in enumerate(data.dtype.names)
}
array_dict

{'time': array(['2018-10-13 11:10:23.560', '2018-10-13 04:34:15.580',
        '2018-10-13 00:13:46.220', '2018-10-12 21:09:49.240',
        '2018-10-12 02:52:03.620'], dtype='<U23'),
 'place': array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
        '42km WNW of Sola, Vanuatu',
        '13km E of Nueva Concepcion, Guatemala',
        '128km SE of Kimbe, Papua New Guinea'], dtype='<U37'),
 'magType': array(['mww', 'mww', 'mww', 'mww', 'mww'], dtype='<U3'),
 'mag': array([6.7, 5.2, 5.7, 5.7, 5.6]),
 'alert': array(['green', 'green', 'green', 'green', 'green'], dtype='<U5'),
 'tsunami': array([1, 0, 0, 0, 1])}

Endi maksimal magnitudani olish faqatgina `mag` kalitini tanlash va `max()` metodini chaqirishdan iborat. Bu faqat 5 ta kiritish bilan ishlaganda list comprehension implementatsiyasidan deyarli ikki baravar tezroq ishlaydi, katta ma'lumotlar to'plamlari bilan ishlaganda birinchi urinish qanchalik yomon ishlashini tasavvur qiling:


In [ ]:
%%timeit
array_dict['mag'].max()

3.18 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Biroq, bu tasvirlashda boshqa muammolar mavjud. Agar biz maksimal magnitudaga ega bo'lgan zilzila haqida barcha ma'lumotlarni olishni xohlasak, qanday qilib buni amalga oshirishimiz mumkin? Biz maksimal qiymatning indeksini topishimiz va keyin lug'atdagi har bir kalit uchun o'sha indeksni olishimiz kerak bo'ladi:


In [ ]:
np.array([
    value[array_dict['mag'].argmax()]
    for key, value in array_dict.items()
])

array(['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
       'mww', '6.7', 'green', '1'], dtype='<U32')

Natija: berilgan indeks bo'yicha qaytarilgan NumPy massividir. Qisqa qilib aytganda ma'lumotlar turlari aralashib kelganda numpy ham biroz oqsoqlanib qoladi va qo'shimcha amallarni bajarishni taqazo etadi. Bu esa ortiqcha resurs degani. Shuning uchun ham Pandasdan foydalanish tavsiya etiladi.

## `Series`
`Series` klassi bitta turdagi massivlar uchun ma'lumot strukturasini va ba'zi qo'shimcha funksionallikni taqdim etadi.


In [ ]:
import pandas as pd

place = pd.Series(array_dict['place'], name='place')
place

0          262km NW of Ozernovskiy, Russia
1              25km E of Bitung, Indonesia
2                42km WNW of Sola, Vanuatu
3    13km E of Nueva Concepcion, Guatemala
4      128km SE of Kimbe, Papua New Guinea
Name: place, dtype: object

Quyida `Series` obyektlari bilan keng foydalaniladigan atributlar keltirilgan:

|Atribut | Qaytaradi |
| --- | --- |
| `name` | `Series` obyektining nomi |
| `dtype` | `Series` obyektining ma'lumot turi |
| `shape` | `Series` obyektining o'lchamlari `(qatorlar soni,)` shaklida |
| `index` | `Series` obyektining tarkibidagi `Index` obyekti |
| `values` | `Series` obyektidagi ma'lumotlar |

Asosan, `pandas` obyektlari ichki ma'lumot tasvirlanishini ta'minlash uchun NumPy massivlaridan foydalanadi. Biroq, ba'zi ma'lumot turlari uchun, `pandas` NumPy ustida qurilib, o'zining [massivlarini](https://pandas.pydata.org/pandas-docs/stable/reference/arrays.html) yaratadi. Shuning uchun, ma'lumot turiga qarab, `values` yoki `pandas.array`, yoki `numpy.array` obyektini qaytarishi mumkin.


In [ ]:
place.name

'place'

### Getting the data type
A `Series` object holds a single data type. Here it is `'O'` for object.

In [ ]:
place.dtype

dtype('O')

### Ma'lumot turini olish
`Series` obyekti bitta ma'lumot turini saqlaydi. Bu yerda u `'O'` - obyekt turi.


In [ ]:
place.shape

(5,)

### Series qiymatlarini ajratish
Ushbu `Series` obyekti qiymatlarini NumPy massiv sifatida saqlamoqda:


In [ ]:
place.values

array(['262km NW of Ozernovskiy, Russia', '25km E of Bitung, Indonesia',
       '42km WNW of Sola, Vanuatu',
       '13km E of Nueva Concepcion, Guatemala',
       '128km SE of Kimbe, Papua New Guinea'], dtype=object)

## `Index`
`Index` klassining qo'shilishi `Series` klassini NumPy massividan ko'ra kuchliroq qiladi. Biz `Series` obyektining `index` atributidan indeksni olishimiz mumkin:


In [ ]:
place_index = place.index
place_index

RangeIndex(start=0, stop=5, step=1)

`Series` obyektlari bilan bir xil tarzda, biz asosiy ma'lumotlarga `values` atributi orqali kirishimiz mumkin. E'tibor bering, ushbu `Index` obyekti ham NumPy massivining ustiga qurilgan:


In [ ]:
place_index.values

array([0, 1, 2, 3, 4], dtype=int64)

Quyida `Index` obyektlari bilan keng tarqalgan foydalaniladigan atributlar keltirilgan:

|Atribut | Qaytaradi |
| --- | --- |
| `name` | `Index` obyektining nomi |
| `dtype` | `Index` obyektining ma'lumot turi |
| `shape` | `Index` obyektining o'lchamlari |
| `values` | `Index` obyektidagi ma'lumotlar |
| `is_unique` | `Index` obyektining barcha qiymatlari unikalligi tekshiriladi |


In [ ]:
place_index.dtype

dtype('int64')

O'lchamini quyidagicha ko'rish mumkin

In [ ]:
place_index.shape

(5,)

Takrorlanmas (unique) qiymatga tekshirish

In [ ]:
place_index.is_unique

True

NumPy bilan biz massivlar orasida elementlarga asoslangan arifmetik amallarni bajarishimiz mumkin:


In [ ]:
np.array([1, 1, 1]) + np.array([-1, 0, 1])

array([0, 1, 2])

Pandas ham buni qo'llab-quvvatlaydi, va indekslari mos elementlarga asoslangan amallarni  bajarilishni imkonini beradi. Qo'shish amali bilan faqat mos keladigan indekslar yig'indisi olinadi:


In [ ]:
numbers = np.linspace(0, 10, num=5) # makes numpy array([0, 2.5, 5, 7.5, 10])
x = pd.Series(numbers) # index is [0, 1, 2, 3, 4]
y = pd.Series(numbers, index=pd.Index([1, 2, 3, 4, 5]))
x + y

0     NaN
1     2.5
2     7.5
3    12.5
4    17.5
5     NaN
dtype: float64

Biz faqat listga o'xshash strukturalarning butun sonli indekslari bilan cheklanmaymiz, va biz qatordagi nomlarni belgilashimiz mumkin. Nomlar istalgan vaqtda o'zgartirilishi mumkin va ular sanalar yoki hatto boshqa biror ustun bo'lishi mumkin.

# `DataFrame`
Har bir ustun uchun `Series` obyektiga ega bo'lish NumPy taqdimotiga nisbatan yaxshilanishdir; ammo, biz hali ham qiymatga asoslangan saralash yoki butun qatordan foydalanish kabi muammolarga duch kelamiz. `DataFrame` bizga bir nechta `Series` obyektlaridan tashkil topgan jadvallarni va qatordagi nomlarni belgilovchi umumiy `Index` obyektini taqdim etadi. Biz `DataFrame` obyekti yaratishimiz mumkin, bu esa ilgari ishlagan NumPy taqdimotlaridan biri asosida amalga oshiriladi (har bir ustun uchun `Series` obyektini yaratishimiz mumkin, ammo bunga hojat yo'q):


In [ ]:
df = pd.DataFrame(array_dict)

# this will also work with the first representation
# df = pd.DataFrame(data)

df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia",mww,6.7,green,1
1,2018-10-13 04:34:15.580,"25km E of Bitung, Indonesia",mww,5.2,green,0
2,2018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu",mww,5.7,green,0
3,2018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala",mww,5.7,green,0
4,2018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea",mww,5.6,green,1


Biz asosiy ma'lumot turini `dtypes` yordamida tekshirishingiz mumkin (diqqat qiling, bu `Series` va `Index` obyektlaridagi `dtype` emas, chunki har bir ustun o'zining ma'lumot turiga ega bo'ladi):


In [ ]:
df.dtypes

time        object
place       object
magType     object
mag        float64
alert       object
tsunami      int32
dtype: object

Biz asosiy ma'lumotlarni `values` atributi yordamida olamiz. Diqqat qiling, bu boshlang'ich NumPy taqdimotimizga juda o'xshash ko'rinadi:


In [ ]:
df.values

array([['2018-10-13 11:10:23.560', '262km NW of Ozernovskiy, Russia',
        'mww', 6.7, 'green', 1],
       ['2018-10-13 04:34:15.580', '25km E of Bitung, Indonesia', 'mww',
        5.2, 'green', 0],
       ['2018-10-13 00:13:46.220', '42km WNW of Sola, Vanuatu', 'mww',
        5.7, 'green', 0],
       ['2018-10-12 21:09:49.240',
        '13km E of Nueva Concepcion, Guatemala', 'mww', 5.7, 'green', 0],
       ['2018-10-12 02:52:03.620', '128km SE of Kimbe, Papua New Guinea',
        'mww', 5.6, 'green', 1]], dtype=object)

Biz ustunlarni `columns` atributi yordamida ajratib olamiz. E'tibor bering, ustunlar aslida `Index` obyekti bo'lib, faqat boshqa o'qi bo'yicha

In [ ]:
df.columns

Index(['time', 'place', 'magType', 'mag', 'alert', 'tsunami'], dtype='object')

Quyidagi atributlar bilan ishlash  keng tarqalgan:

|Atribut | Natija |
| --- | --- |
| `dtypes` | Har bir ustunning ma'lumot turlari |
| `shape` | `DataFrame` obyekti o'lchamlari, `(satrlar soni, ustunlar soni)` shaklida |
| `index` | `DataFrame` obyekti bo'ylab satrlar uchun `Index` obyekti |
| `columns` | Ustunlar nomi ( `Index` obyekti sifatida) |
| `values` | `DataFrame` obyekti ma'lumotlari |
| `empty` | `DataFrame` obyekti bo'sh ekanligini tekshirish |

`DataFrame` obyekti bo'ylab satrlar uchun `Index` obyekti `index` atributi orqali olinishi mumkin (bu `Series` ob'ektlari kabi):


In [ ]:
df.index

RangeIndex(start=0, stop=5, step=1)

`Series` va `Index` ob'ektlari kabi, `DataFrame` obyektining o'lchamlarini `shape` atributi orqali olishimiz mumkin. Natija `(satrlar soni, ustunlar soni)` shaklida bo'ladi. Bizning `DataFrame` obyektimizda 5 satr va 6 ustun mavjud:


In [ ]:
df.shape

(5, 6)

Shuni yodda tutingki, `DataFrame` ob'ektlari bilan arifmetik amallarni ham bajarishimiz mumkin. Pandas faqat indeks va ustun mos kelsa, amallarni bajaradi. Bu yerda qo'shish amali ko'rsatilgan. Stringlar bilan qo'shish esa ulashishni anglatadi, shuning uchun `pandas` satrlar (`time`, `place`, `magType`, va `alert`) bo'yicha qo'shish amalga oshirdi. Raqamli ustunlar (`mag` va `tsunami`) yig'indisini hisoblagan:


In [ ]:
df + df

,time,place,magType,mag,alert,tsunami
0,2018-10-13 11:10:23.5602018-10-13 11:10:23.560,"262km NW of Ozernovskiy, Russia262km NW of Oze...",mwwmww,13.4,greengreen,2
1,2018-10-13 04:34:15.5802018-10-13 04:34:15.580,"25km E of Bitung, Indonesia25km E of Bitung, I...",mwwmww,10.4,greengreen,0
2,2018-10-13 00:13:46.2202018-10-13 00:13:46.220,"42km WNW of Sola, Vanuatu42km WNW of Sola, Van...",mwwmww,11.4,greengreen,0
3,2018-10-12 21:09:49.2402018-10-12 21:09:49.240,"13km E of Nueva Concepcion, Guatemala13km E of...",mwwmww,11.4,greengreen,0
4,2018-10-12 02:52:03.6202018-10-12 02:52:03.620,"128km SE of Kimbe, Papua New Guinea128km SE of...",mwwmww,11.2,greengreen,2


<hr>
<div>
    <a href="../ch_01/introduction_to_data_analysis.ipynb">
        <button style="float: left;">&#8592; Chapter 1</button>
    </a>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>